# Tests on parts of  NA12878 data for Del_st, Del_end and No_sv, ground  truth Mills_nanosv

We are going to use a network trained on a random subsample of NoSv, and apply it to the whole training set. Then we are going to take the confused NoSvs as training Nosvs to a nother network
The training set is the first 16 chromozomzes, the test is the rest

## Load dependences and setting output configuration

In [1]:
import numpy as np
from keras.utils.np_utils import to_categorical
import keras
%pylab inline
%load_ext autoreload
%autoreload 2

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Populating the interactive namespace from numpy and matplotlib


## Load data from npy files

In [2]:
import pickle
dico = pickle.load(open("/home/ubuntu/GTCG_labels/labels.pickle", "rb"))
data_labels = dataset = dico["Mills2011_nanosv"]
window_size = 200

In [3]:
dico = []

In [4]:
replications = 16
factor_nosv = 1 

In [5]:
# after iteration 4 I was taking only the SV windows because of memory problems and because I don't really need all nosv

In [6]:
datapath = '/home/ubuntu/GTCG_data' 


germline_1_datapath = datapath + '/'
#ger_1_labels_datapath = datapath + '/LabelData/unzipped/'

G1_data_SV = np.empty([0, 27, window_size])
G1_labels_SV =[]
G1_data_noSV = np.empty([0, 27, window_size])
G1_labels_noSV =[]
total_train_sv_windows = 0
for i in range(1,16,1):
        G1_data_new =np.load(germline_1_datapath + str(i)  +'_channel_maker_real_germline.npy')
        #G1_labels_new = np.load(ger_1_labels_datapath  +  str(i) +'_label_ci_full_overlap.npy/' +str(i) +'_label_ci_full_overlap.npy') 
        G1_labels_new = np.array(data_labels[str(i)])
        print(G1_labels_new[0])
        G1_data_new = G1_data_new[np.where(G1_labels_new != 'UK')]
        G1_labels_new = G1_labels_new[np.where(G1_labels_new != 'UK')]
        G1_data_SV_new  = G1_data_new[np.where(G1_labels_new != 'noSV')]
        G1_labels_SV_new  = G1_labels_new[np.where(G1_labels_new != 'noSV')]
        n_windows_sv = G1_data_SV_new.shape[0]
        G1_data_noSV_new  = G1_data_new[np.where(G1_labels_new == 'noSV')]
        G1_labels_noSV_new  = G1_labels_new[np.where(G1_labels_new == 'noSV')]
        #G1_data_noSV_new =  G1_data_noSV_new[0:int(replications*n_windows_sv/2/factor_nosv)]
        #G1_labels_noSV_new =  G1_labels_noSV_new[0:int(replications*n_windows_sv/2/factor_nosv)]
        print(str(i) + ':' + str(G1_data_SV_new.shape))
        #print(G1_labels_SV_new[0])
        G1_data_SV = np.concatenate((G1_data_SV, G1_data_SV_new))
        G1_labels_SV = np.concatenate((G1_labels_SV, G1_labels_SV_new))
        G1_data_noSV = np.concatenate((G1_data_noSV, G1_data_noSV_new))
        G1_labels_noSV = np.concatenate((G1_labels_noSV, G1_labels_noSV_new))
        if i < 16: #for training until chromozome 16
            total_train_sv_windows += G1_data_SV_new.shape[0]

noSV
1:(222, 27, 200)
noSV
2:(205, 27, 200)
noSV
3:(208, 27, 200)
noSV
4:(228, 27, 200)
noSV
5:(214, 27, 200)
noSV
6:(162, 27, 200)
noSV
7:(163, 27, 200)
noSV
8:(135, 27, 200)
noSV
9:(127, 27, 200)
noSV
10:(107, 27, 200)
noSV
11:(119, 27, 200)
noSV
12:(114, 27, 200)


MemoryError: 

### Find  the indexes that split the categories

In [ ]:
def find_splitting_indices(labels):
    for i in range (0, labels.shape[0] - 1):
        if (labels[i] != labels[i+1]):
            print(i , ': ', labels[i] , labels[i+1] )        

In [ ]:
#find_splitting_indices(G1_labels)

In [ ]:
#G1_labels_SV = ['DEL'] * G1_labels_SV.shape[0]

In [ ]:
sv=G1_data_SV.shape[0]

In [ ]:
sv

In [ ]:
int(sv/2)

In [ ]:
#G1_data_noSV  = G1_data_noSV[0:2*sv] 

In [ ]:
#G1_labels_noSV  = G1_labels_noSV[0:2*sv] 

In [ ]:
SV_train =  0 #int(0.8 * total_train_sv_windows)
noSV_train = 0 #int(replications * SV_train / 2/factor_nosv)

SV_val = 0 # total_train_sv_windows - SV_train
noSV_val = 0 # int(replications * SV_val / 2/factor_nosv)

In [ ]:
def replicate(data, start, end, howmanytimes): 
    X = data[start:end]
    for i in range(0,howmanytimes):
        X = np.concatenate((X, data[start:end]))
    return X 

In [ ]:
replications = 1#no rep for testing
X_train = replicate(G1_data_SV, 0, SV_train, replications)
y_train = replicate(G1_labels_SV, 0, SV_train,replications)
X_val = replicate(G1_data_SV, SV_train, SV_train+SV_val, replications)
y_val = replicate(G1_labels_SV, SV_train, SV_train+SV_val, replications)
X_test = replicate(G1_data_SV, SV_train+SV_val, len(G1_data_SV), replications)
y_test = replicate(G1_labels_SV, SV_train+SV_val, len(G1_labels_SV), replications)

In [ ]:
G1_data_noSV.shape

In [ ]:
G1_data_noSV = G1_data_noSV[0:200000] 

In [ ]:
G1_labels_noSV = G1_labels_noSV[0:200000] 

In [ ]:
X_train = np.concatenate((X_train, G1_data_noSV[0:noSV_train ]))
X_val = np.concatenate((X_val, G1_data_noSV[noSV_train:noSV_train +noSV_val ]))
X_test = np.concatenate((X_test, G1_data_noSV[noSV_train +noSV_val:]))

In [ ]:
G1_data_noSV = []

In [ ]:
y_train = np.concatenate((y_train, G1_labels_noSV[0:noSV_train ]))
y_val = np.concatenate((y_val, G1_labels_noSV[noSV_train:noSV_train +noSV_val ]))
y_test = np.concatenate((y_test, G1_labels_noSV[noSV_train +noSV_val:]))

In [ ]:
#G1_labels_noSV = []

In [ ]:
y_test

### Number of training, validation and test windows

In [ ]:
X_train.shape

In [ ]:
X_val.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

In [ ]:
number_channels = X_train.shape[1]
number_channels

## Normalize data using the reads coverage as parameter and remove uninteresting channels for deletions

In [ ]:
coverage = np.median(X_test[:, 0, :]) #the median of the coverage in the normal cells
coverage


In [ ]:
def normalize(X):       
    for i in range (0, len(X)):                              
        for j in range(0,number_channels):                                                                                            
            X[i][j] = np.divide(X[i][j], coverage)*100 
normalize(X_train)
normalize(X_val)
normalize(X_test)

In [ ]:
def eliminate_noise_chanels(X):
    return np.delete(X, [4-1,5-1,6-1,7-1,8-1,9-1,10-1,12-1,13-1,15-1,16-1, 17-1,18-1,19-1,23-1,24-1,26-1,27-1], axis=1)
    

In [ ]:
def eliminate_distance_chanels(X):
    return np.delete(X, [3,4,7,8], axis=1)

In [ ]:
X_train = eliminate_noise_chanels(X_train)
X_val = eliminate_noise_chanels(X_val)
X_test = eliminate_noise_chanels(X_test)

In [ ]:
X_train = eliminate_distance_chanels(X_train)
X_val = eliminate_distance_chanels(X_val)
X_test = eliminate_distance_chanels(X_test)

In [ ]:
X_train.shape

In [ ]:
number_channels = X_train.shape[1]
number_channels

## Differentiate the time series for a zero mean

In [ ]:
def differentiate(X):
    for i in range(0, X.shape[0]):        
        for j in range(0, X.shape[1]):
            for k in range(0, X.shape[2]-1):
                X[i][j][k] = X[i][j][k+1] - X[i][j][k]
    return X[:,:,:-1]        

In [ ]:
#X_train = differentiate(X_train)

In [ ]:
#X_val = differentiate(X_val)
#X_test = differentiate(X_test)

## Plots several  windows

In [ ]:
n_windows = 100
label = ["None"] * number_channels

In [ ]:
label[0] = "germline:coverage"
label[1] = "germline:#left clipped reads"
label[2] = "germline:#right clipped reads"
label[3] = "germline:#split reads right split"
label[4] = "germline:#split reads left split"

In [ ]:
from numpy import *
import math
import matplotlib.pyplot as plt

colors =['b','g', 'm', 'gray', 'r','c', 'darkgreen' ,'y', 'k', 'orange', 'gray']

def plot_channels(start_window, n_windows):
    for i in range (start_window, start_window + n_windows, 1):
        if labels[y_test[i]] == labels[predicted[i]]:
            continue
        #print(y_train[i], 'id:', i)
        print('id:', i)
        print('Nanopore:', labels[y_test[i]], '. DeepSV:', labels[predicted[i]])
        for j in range (0, number_channels):   
            shift = 0
            start =0
            if j in [0,11]:
                shift = -100
            if j > 8:
                start = -60
            Z = [start + shift + 300 + x - 5*j*4 for x in X_test[i][j]]  
            plt.ylim([150,450])
            if np.max(Z) > 1000:
                plt.ylim([-200,1000])
                Z = [start + shift + 300 + x - 5*j*4 - 400 for x in X_test[i][j]]  
            if j>8:
                plt.plot(Z,  label = label[j], linestyle=':', linewidth=1.5, color = colors[j % 9])
            else:
                plt.plot(Z,  label = label[j], color = colors[j])
        
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., prop={'size': 10})    
        plt.show()

In [ ]:
start_window = 0
n_windows = 5000
plot_channels(start_window, n_windows)

In [ ]:
n_windows = 1000
start_window = 10500
plot_channels(start_window, n_windows)

   
## Transposes every window in X, to comply to McFly format

In [ ]:
def transposeDataset(X):
    image = []
    for i in range (0, len(X -1)):
        tr = X[i].transpose()
        image.append(tr)
    return np.array(image)

image1 = transposeDataset(X_train)
image2 = transposeDataset(X_val)
image3 = transposeDataset(X_test)
X_train = image1
X_val = image2
X_test = image3

n_channels = len(X_test[0][0])
print(len(X_test[0][0]))

## Prepare the labels in McFly format

In [ ]:
mapclasses = {'DEL_start': 1, 'DEL_end': 0,  'noSV': 2}
print(mapclasses)

In [ ]:
#y_train = np.array([mapclasses[c] for c in y_train], dtype='int')
#y_val = np.array([mapclasses[c] for c in y_val], dtype='int')
y_test = np.array([mapclasses[c] for c in y_test], dtype='int')
#y_train_binary = to_categorical(y_train)
#y_val_binary = to_categorical(y_val)
y_test_binary = to_categorical(y_test)

In [ ]:
y_test_binary

In [ ]:
X_train.shape

## Reload the trained model

In [ ]:
import os
from keras.models import load_model
model_path = os.path.join('/home/ubuntu/CNN/notebooks/',  'Gtcg_01_11_32xoversample_NoDiff_9715')
model_reloaded = load_model(model_path)

## Inspect model predictions on test data

In [ ]:

model = model_reloaded 
datasize = X_test.shape[0]
probs = model.predict_proba(X_test[:datasize,:,:],batch_size=1)

## Columns are predicted labels by DeepSV, rows are nanopore labels

In [ ]:
mapclasses = {'DEL_end': 0, 'DEL_start': 1, 'noSV': 2}
dict=mapclasses
dict_sorted = sorted(dict.items(), key=lambda x: x[1])
dict_sorted

In [ ]:
labels = [i[0] for i in dict_sorted]

In [ ]:
import pandas as pd
predicted = probs.argmax(axis=1)
y_index = y_test_binary.argmax(axis=1)
confusion_matrix = pd.crosstab(pd.Series(y_index), pd.Series(predicted))
confusion_matrix.index = [labels[i] for i in confusion_matrix.index]
confusion_matrix.columns = [labels[i] for i in confusion_matrix.columns]
confusion_matrix.reindex(columns=[l for l in labels], fill_value=0)
confusion_matrix

## Test best current model on testset

### Overall accuracy on the test set:

In [ ]:
%%time
score_test = model.evaluate(X_test, y_test_binary, verbose=False)
print('Test loss and accuracy of best model: ' + str(score_test))

In [ ]:
#9659 best test results : 9513

In [ ]:
4000/25


In [ ]:
 print('Nanopore:', labels[y_test[i]], '. DeepSV:', labels[predicted[i]])

In [ ]:
G1_data_noSV_confused = X_test[np.where(y_test == 2)]

In [ ]:
G1_labels_noSV_confused = y_test[np.where(y_test == 2)]

In [ ]:
predicted_confused = predicted[np.where(y_test == 2)]

In [ ]:
G1_data_noSV_confused = G1_data_noSV_confused[np.where(predicted_confused != 2)]

In [ ]:
G1_labels_noSV_confused = G1_labels_noSV_confused[np.where(predicted_confused != 2)]

In [ ]:
predicted_confused = predicted_confused[np.where(predicted_confused != 2)]

In [ ]:
G1_data_noSV_confused.shape

In [ ]:
G1_labels_noSV_confused.shape

In [ ]:
predicted_confused.shape 

In [ ]:
np.save(germline_1_datapath + "G1_data_noSV_confused.npy", G1_data_noSV_confused)

In [ ]:
np.save(ger_1_labels_datapath + "G1_labels_noSV_confused.npy", G1_labels_noSV_confused)